In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determin

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY_1')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo' # You can change this to other GPT versions

In [ ]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

*   By not setting allow_delegation=False, allow_delegation takes its default value of being True.
*   This means the agent can delegate its work to another agent which is better suited to do a particular task.




In [ ]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)



*   Role Playing: Both agents have been given a role, goal and backstory.
*   Focus: Both agents have been prompted to get into the character of the roles they are playing.
*   Cooperation: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.






In [ ]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

**Possible Custom Tools**

*   Load customer data
*   Tap into previous conversations
*   Load data from a CRM
*   Checking existing bug reports
*   Checking existing feature requests
*   Checking ongoing tickets
... and more

Some ways of using CrewAI tools.
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
Instantiate a document scraper tool.
The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [ ]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [ ]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [ ]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [ ]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

In [ ]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to provide Andrew Ng from DeepLearningAI with the best possible support by thoroughly understanding how to set up a Crew and add memory to it.

Action: Read website content
Action Input: {"url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"} 

Introduction - CrewAICrewAI home pageEnglishSearch...⌘KAsk AIStart Cloud TrialcrewAIInc/crewAIcrewAIInc/crewAISearch...NavigationGet StartedIntroductionDocumentationEnterpriseAPI ReferenceExam

In [ ]:
from IPython.display import Markdown
Markdown(result)

Hello Andrew Ng,

Thank you for reaching out to crewAI for assistance with adding memory to your Crew. We understand the importance of optimizing memory allocation for your AI agents, and we are here to guide you through the process.

To effectively add memory to your Crew in crewAI, follow these steps:

1. Create Your Crew: Begin by creating a Crew in crewAI. A Crew is a collaboration of AI agents with specific roles and goals, working together to accomplish tasks efficiently.

2. Specify Memory Requirements: When setting up your Crew, define the memory requirements for each AI agent based on their tasks and objectives. This ensures that each agent has sufficient memory allocation to perform effectively.

3. Configure Task Execution: Optimize memory usage by configuring task execution within your Crew. You can set up workflows, manage task dependencies, and ensure efficient memory utilization for seamless operations.

4. Monitor and Adjust: Once your Crew is up and running, monitor memory usage and performance of each AI agent. Make adjustments to memory allocation as needed to enhance overall efficiency and productivity.

For more detailed guidance and step-by-step instructions, refer to our documentation on creating a Crew and kicking it off at https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/.

If you have any further questions or require additional assistance with adding memory to your Crew in crewAI, please feel free to reach out to us. Our support team is dedicated to ensuring that you have a smooth and successful experience with our platform.

Thank you for choosing crewAI for your AI needs. We are excited to help you optimize memory allocation for your Crew and enhance your AI operations.

Best regards,

[Your Name]
Senior Support Representative
crewAI

**Tools for a Customer Outreach Campaign**

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY_1')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo' # You can change this to other GPT versions
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [ ]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

**Creating Tools**

In [ ]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [ ]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

**CUstom Tool**

In [ ]:
from crewai_tools import BaseTool

Every Tool needs to have a name and a description.
For simplicity and classroom purposes, SentimentAnalysisTool will return positive for every text.
When running locally, you can customize the code with your logic in the _run function.

In [ ]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

Every Tool needs to have a name and a description.
For simplicity and classroom purposes, SentimentAnalysisTool will return positive for every text.
When running locally, you can customize the code with your logic in the _run function.

In [ ]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [ ]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [ ]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [ ]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],

    verbose=2,
	memory=True
)

In [ ]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about DeepLearningAI to create a comprehensive profile.

Action: Read a file's content
Action Input: {"file_path": "./instructions/DeepLearningAI_profile.txt"} 

I encountered an error while trying to use the tool. This was the error: [Errno 2] No such file or directory: './instructions/DeepLearningAI_profile.txt'.
 Tool Read a file's content accepts these 

In [ ]:
from IPython.display import Markdown
Markdown(result)

Subject: Elevating DeepLearningAI's Success with CrewAI's Innovative Solutions

Dear Andrew Ng,

I hope this message finds you well. I wanted to take this opportunity to extend my congratulations on DeepLearningAI's recent achievements and share how CrewAI's solutions can support your future goals.

As a global leader in AI education, DeepLearningAI has made significant strides in empowering learners worldwide through world-class education and hands-on training. Your recent launch of online courses on deep learning and AI, reaching over 7 million learners, is truly commendable. It reflects your commitment to democratizing AI education and fostering a skilled global workforce for the future.

At CrewAI, we are equally passionate about leveraging cutting-edge tools and strategic insights to optimize course content and delivery. Our personalized engagement strategies and AI-powered tools can enhance DeepLearningAI's course offerings with the latest trends, ensuring that learners have access to the most relevant and impactful educational resources.

We understand the importance of expanding your reach to a wider audience and fostering a strong community of AI enthusiasts. By partnering with CrewAI, DeepLearningAI can benefit from our data-driven insights and personalized communication strategies to engage learners effectively and build a vibrant community of AI enthusiasts.

I believe that our collaboration can elevate DeepLearningAI's success even further, enabling you to achieve your long-term goal of democratizing the creation of AI and empowering more individuals to build custom AI systems.

I would love the opportunity to discuss how CrewAI's solutions can align with DeepLearningAI's objectives and contribute to your continued success. Please let me know if you would be interested in exploring this partnership further.

Thank you for considering CrewAI as a strategic partner in your mission to shape the future of Artificial Intelligence.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI